# 데이터 가공

In [1]:
from utils import  load_musicxml

def play_music(music_data):
    """
    불러온 악보 데이터를 MIDI로 재생합니다.

        입력 : 음악 데이터
        출력 : 음악
    """
    music_data.show('midi')


file_name = '/home/hew0920/Gukak/KTraditionalAIComposer/data/01 G-Sangnyeongsan_Haegeum_part(0807).musicxml.xml'
print('file_name :', file_name, '\n')
music_data = load_musicxml(file_name)
play_music(music_data)

file_name : /home/hew0920/Gukak/KTraditionalAIComposer/data/01 G-Sangnyeongsan_Haegeum_part(0807).musicxml.xml 



### 토큰(Token)화

In [2]:
from data_processing import music_to_tokens

tokens = music_to_tokens(file_name)
print("number of tokens:", len(tokens[0]))
print("Pitch:", tokens[0])
print("Octave:", tokens[1])
print("Duration:", tokens[2])

number of tokens: 274
Pitch: ['<SOS>', 'rest', 'rest', 'rest', 'rest', 'E-', 'A-', 'A-', 'E-', 'B-', 'B-', 'A-', 'E-', 'F', 'A-', 'A-', 'E-', 'B-', 'B-', 'A-', 'A-', 'E-', 'E-', 'F', 'A-', 'A-', 'F', 'B-', 'F', 'E-', 'E-', 'A-', 'E-', 'A-', 'B-', 'E-', 'B-', 'D-', 'E-', 'A-', 'B-', 'D-', 'E-', 'E-', 'F', 'A-', 'A-', 'E-', 'F', 'B-', 'A-', 'B-', 'F', 'F', 'E-', 'E-', 'F', 'A-', 'B-', 'D-', 'E-', 'A-', 'F', 'E-', 'A-', 'E-', 'E-', 'B-', 'D-', 'B-', 'D-', 'B-', 'B-', 'A-', 'E-', 'F', 'B-', 'A-', 'B-', 'F', 'E-', 'B-', 'A-', 'B-', 'F', 'E-', 'A-', 'F', 'E-', 'A-', 'E-', 'E-', 'B-', 'D-', 'B-', 'B-', 'B-', 'A-', 'B-', 'D-', 'B-', 'B-', 'A-', 'B-', 'A-', 'E-', 'E-', 'F', 'A-', 'A-', 'A-', 'E-', 'B-', 'A-', 'A-', 'B-', 'E-', 'B-', 'D-', 'E-', 'A-', 'B-', 'D-', 'E-', 'E-', 'F', 'A-', 'A-', 'E-', 'F', 'B-', 'A-', 'B-', 'F', 'F', 'E-', 'E-', 'F', 'A-', 'B-', 'D-', 'E-', 'A-', 'F', 'E-', 'A-', 'E-', 'E-', 'A-', 'B-', 'B-', 'E-', 'B-', 'A-', 'B-', 'F', 'E-', 'F', 'A-', 'B-', 'D-', 'E-', 'B-', 'B-'

## 사전 처리된 데이터 불러오기

In [6]:
group_number = 2

import pickle
# 파일에서 데이터 불러오기
with open(f'/home/hew0920/Gukak/KTraditionalAIComposer/data/group{group_number}_song_information.pkl', 'rb') as f:
    Song_informations = pickle.load(f)

In [7]:
Song_informations[0].keys()

dict_keys(['Song_name', 'Song_pitch', 'Song_octave', 'Song_duration'])

In [8]:
from data_processing import build_predefined_vocabs, build_vocab_mapping, music_to_tokens
from dataset import GukAkDataset

# 사전 정의된 vocabularies 생성
pitch_vocab, octave_vocab, duration_vocab = build_predefined_vocabs()
pitch_token_to_idx, pitch_idx_to_token = build_vocab_mapping(pitch_vocab)
octave_token_to_idx, octave_idx_to_token = build_vocab_mapping(octave_vocab)
duration_token_to_idx, duration_idx_to_token = build_vocab_mapping(duration_vocab)

dataset = GukAkDataset(Song_informations=Song_informations,
                       pitch_token_to_idx=pitch_token_to_idx,
                       octave_token_to_idx=octave_token_to_idx,
                       duration_token_to_idx=duration_token_to_idx,
                       pad=False,
                       max_length=None)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def visualize_vocab(vocab, title="Vocabulary"):
    """
    vocab: 토큰의 리스트 (예: pitch_vocab)
    title: 플롯 제목
    """
    # DataFrame 생성: 인덱스와 토큰을 열로 구성
    df = pd.DataFrame({
        "Index": list(range(len(vocab))),
        "Token": vocab
    })

    # Figure 생성 (세로 길이를 토큰 수에 맞게 조정)
    fig, ax = plt.subplots(figsize=(4, len(vocab)*0.3 + 1))
    ax.axis('tight')
    ax.axis('off')

    # DataFrame의 내용을 테이블로 표시
    table = ax.table(cellText=df.values, colLabels=df.columns, loc="center", cellLoc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 1.5)

    plt.title(title)
    plt.show()

# 미리 정의된 vocabulary 예시 (이전에 build_predefined_vocabs()로 생성한 값)
pitch_vocab, octave_vocab, duration_vocab = build_predefined_vocabs()

# 시각화 실행
visualize_vocab(pitch_vocab, "Pitch Vocabulary")
visualize_vocab(octave_vocab, "Octave Vocabulary")
visualize_vocab(duration_vocab, "Duration Vocabulary")


In [11]:
from torch.utils.data import DataLoader

data_loader = DataLoader(dataset, batch_size=1, shuffle=True)

# Model 설정 및 학습코드

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from model import GukAkLSTM
from train import train

# Hyperparameters
n_epochs = 200
lr = 0.01
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pitch_vocab_size = len(pitch_vocab)
octave_vocab_size = len(octave_vocab)
duration_vocab_size = len(duration_vocab)

model = GukAkLSTM(pitch_vocab_size, octave_vocab_size, duration_vocab_size).to(device)


optimizer = optim.Adam(model.parameters(), lr=lr)

# 각 채널별로 <PAD> 토큰 인덱스를 ignore_index로 지정합니다.
pitch_ignore_index = pitch_token_to_idx["<PAD>"]
octave_ignore_index = octave_token_to_idx["<PAD>"]
duration_ignore_index = duration_token_to_idx["<PAD>"]

criterion_pitch = nn.CrossEntropyLoss(ignore_index=pitch_ignore_index)
criterion_octave = nn.CrossEntropyLoss(ignore_index=octave_ignore_index)
criterion_duration = nn.CrossEntropyLoss(ignore_index=duration_ignore_index)

In [13]:
for epoch in range(n_epochs):
    model.train()
    total_loss = 0.0

    for batch in data_loader:
        pitch_batch, octave_batch, duration_batch = batch

        # 텐서를 device로 이동 (각 텐서 shape: [batch_size, max_length])
        pitch_batch = pitch_batch.to(device)
        octave_batch = octave_batch.to(device)
        duration_batch = duration_batch.to(device)

        # 입력과 타겟 시퀀스 구성:
        # 입력: 시퀀스의 처음 ~ (T-1)번째 토큰 [<SOS>, first_note, ...., last_note]
        # 타겟: 시퀀스의 1번째 ~ T번째 토큰 (즉, 한 스텝 뒤의 정답) [first_note, ..., last_note, <EOS>]
        input_pitch = pitch_batch[:, :-1]      # (batch_size, max_length-1)
        target_pitch = pitch_batch[:, 1:]      # (batch_size, max_length-1)

        input_octave = octave_batch[:, :-1]
        target_octave = octave_batch[:, 1:]

        input_duration = duration_batch[:, :-1]
        target_duration = duration_batch[:, 1:]

        optimizer.zero_grad()

        # 모델에 입력을 전달하여 각 채널별 예측 로짓을 얻습니다.
        # 각 출력의 shape:
        #   output_pitch: (batch_size, seq_len, pitch_vocab_size)
        #   output_octave: (batch_size, seq_len, octave_vocab_size)
        #   output_duration: (batch_size, seq_len, duration_vocab_size)
        output_pitch, output_octave, output_duration, _ = model(input_pitch, input_octave, input_duration)

        loss_pitch = criterion_pitch(output_pitch.view(-1, pitch_vocab_size), target_pitch.view(-1))
        loss_octave = criterion_octave(output_octave.view(-1, octave_vocab_size), target_octave.view(-1))
        loss_duration = criterion_duration(output_duration.view(-1, duration_vocab_size), target_duration.view(-1))

        loss = loss_pitch + loss_octave + loss_duration
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(data_loader)
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {avg_loss:.4f}")

Epoch 1/200, Loss: 6.4295
Epoch 2/200, Loss: 4.9723
Epoch 3/200, Loss: 4.8569
Epoch 4/200, Loss: 4.8243
Epoch 5/200, Loss: 4.8424
Epoch 6/200, Loss: 4.7760
Epoch 7/200, Loss: 4.6703
Epoch 8/200, Loss: 4.5931
Epoch 9/200, Loss: 4.5521
Epoch 10/200, Loss: 4.5189
Epoch 11/200, Loss: 4.4724
Epoch 12/200, Loss: 4.4065
Epoch 13/200, Loss: 4.3388
Epoch 14/200, Loss: 4.2761
Epoch 15/200, Loss: 4.1974
Epoch 16/200, Loss: 4.1601
Epoch 17/200, Loss: 4.0914
Epoch 18/200, Loss: 4.0588
Epoch 19/200, Loss: 3.9963
Epoch 20/200, Loss: 3.9482
Epoch 21/200, Loss: 3.9174
Epoch 22/200, Loss: 3.8779
Epoch 23/200, Loss: 3.8426
Epoch 24/200, Loss: 3.8360
Epoch 25/200, Loss: 3.7799
Epoch 26/200, Loss: 3.7649
Epoch 27/200, Loss: 3.7280
Epoch 28/200, Loss: 3.6943
Epoch 29/200, Loss: 3.6776
Epoch 30/200, Loss: 3.6709
Epoch 31/200, Loss: 3.6113
Epoch 32/200, Loss: 3.5827
Epoch 33/200, Loss: 3.5530
Epoch 34/200, Loss: 3.5191
Epoch 35/200, Loss: 3.5004
Epoch 36/200, Loss: 3.4788
Epoch 37/200, Loss: 3.4548
Epoch 38/2

In [14]:
from generate_music import generate_musics, remove_special_tokens

# 예시 사용:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generated_music = generate_musics(model,
                                pitch_token_to_idx, octave_token_to_idx, duration_token_to_idx,
                                pitch_idx_to_token, octave_idx_to_token, duration_idx_to_token,
                                device, max_gen_length=300)
                                
gen_pitch, gen_octave, gen_duration = remove_special_tokens(generated_music)

print("Generated Pitch Sequence:", gen_pitch)
print("Generated Octave Sequence:", gen_octave)
print("Generated Duration Sequence:", gen_duration)

Generated Pitch Sequence: ['rest', 'A-', 'D-', 'B-', 'A-', 'D-', 'B-', 'A-', 'G-', 'B-', 'E-', 'D-', 'B-', 'G-', 'B-', 'E-', 'D-', 'G-', 'E-', 'B-', 'D-', 'B-', 'A-', 'B-', 'D-', 'A-', 'B-', 'G-', 'E-', 'rest', 'F', 'A-', 'B-', 'G-', 'G-', 'D-', 'A-', 'B-', 'E-', 'B-', 'A-', 'B-', 'E-', 'C', 'F', 'E-', 'A-', 'B-', 'G-', 'B-', 'E-', 'B-', 'D-', 'A-', 'A-', 'D-', 'A-', 'rest', 'rest', 'B-', 'E-', 'G-', 'B-', 'E-', 'B-', 'E-', 'D-', 'A-', 'E-', 'G-', 'E-', 'G-', 'E-', 'A-', 'A-', 'A-', 'B-', 'B-', 'E-', 'B-', 'B-', 'B-', 'G-', 'A-', 'B-', 'E-', 'A-', 'G-', 'F', 'B-', 'A-', 'B-', 'A-', 'rest', 'E-', 'A-', 'B-', 'E-', 'A-', 'B-', 'B-', 'B-', 'B-', 'B-', 'G-', 'A-', 'B-', 'D-', 'A-', 'D-', 'A-', 'B-', 'D-', 'A-', 'B-', 'A-', 'G-', 'B-', 'D-', 'F', 'E-', 'A-', 'B-', 'D-', 'A-', 'B-', 'A-', 'rest', 'B-', 'A-', 'G-', 'A-', 'G-', 'A-', 'E-', 'G-', 'E-', 'rest', 'B-', 'E-', 'E-', 'G-', 'B-', 'D-', 'A-', 'A-', 'rest', 'A-', 'A-', 'B-', 'C', 'A-', 'F', 'B-', 'B-', 'G-', 'E-', 'E-', 'G-', 'B-', 'G-'

/home/hew0920/Gukak/KTraditionalAIComposer/generate_music.py:225: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  for ind in list(np.where((np.array(gen_pitch) =='<EOS>')| (np.array(gen_pitch) =='<PAD>')| (np.array(gen_pitch) =='<SOS>'))[0])+ list(np.where( (np.array(gen_octave) == 0 )| (np.array(gen_octave) == '0' )| (np.array(gen_octave) =='<EOS>')| (np.array(gen_octave) =='<PAD>')| (np.array(gen_octave) =='<SOS>'))[0]) +  list(np.where( (np.array(gen_duration) =='<EOS>')| (np.array(gen_duration) =='<PAD>')| (np.array(gen_duration) =='<SOS>'))[0]):


In [15]:
def save_stream_as_midi(music_stream, file_path):
    """
    music21 stream 객체를 MIDI 파일로 저장합니다.

    인자:
      - music_stream: music21의 stream 객체
      - file_path: 저장할 MIDI 파일 경로 (예: "generated_music.mid")
    """
    music_stream.write('midi', fp=file_path)

def save_stream_as_musicxml(music_stream, file_path):
    """
    music21 stream 객체를 MusicXML 파일로 저장합니다.

    인자:
      - music_stream: music21의 stream 객체
      - file_path: 저장할 MusicXML 파일 경로 (예: "generated_music.musicxml")
    """
    music_stream.write('musicxml', fp=file_path)

In [20]:
from data_processing import tokens_to_music21

cutting_number = 4 
time_signature = f'{3*cutting_number}/8'
info_tokens = [gen_pitch, gen_octave, gen_duration]

new_music = tokens_to_music21(info_tokens, time_signature= time_signature, cutting_number = cutting_number)

save_stream_as_midi(new_music, 'GukAkLSTM.midi')
save_stream_as_musicxml(new_music, 'GukAkLSTM.musicxml')

In [24]:
# Print score
print('installing lilypond...')
!apt-get install lilypond > /dev/null

import time
from IPython.display import Image

def show(music):
  fpath = str(music.write('lily.png'))
  time.sleep(10)
  display(Image(fpath))

installing lilypond...
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [1]:
# from music21 import converter
# loaded_music = converter.parse('GukAkLSTM.musicxml')
# show(loaded_music)

In [ ]:
new_music.show('midi')